<a href="https://colab.research.google.com/github/flydevilz/automate_zoom_attendance/blob/main/Check_Presensi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Use:
1. Download CSV di ZOOM, dan **jangan centang Export Data Meeting**. Setelah itu masukkan csv di GDrive masing2.
2. Set share -> general access -> anyone with the link
3. Copy link dan paste di bagian Import Data -> `link attendance`. Ubah ke section masing-masing.
4. Runtime -> Run All
5. Yg ditutup gausah dibuka yaa, yg dibutuhkan cuma bagian "Tambah Presensi", "Tidak Hadir" dan "Ga ke detect" yaa((:

Note: untuk bagian  `link attendance` kalian pakai file terpisah dari csv yg di programnya ka Dew (soalnya yg di spreadsheet kan downloadnya file csv zoom nya pakai export data meeting).
Program disini baru bisa jalan kalau csv nya gak di export data meetingnya (:
Jadi kalau mau pake ini harus download lg file csv zoom nya yg gk pakai export data meeting (yg disuru centang kemaren, jgn di centang). Tunggu update patch berikutnya ygy baru bisa hehe(:


# Import Data

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
insert_link = str(input())

https://drive.google.com/file/d/1qDd7S6z9uu4_Elr2I5nHRzL8Qk3jFHgB/view?usp=drive_link


In [3]:
pattern_link = r"https://drive\.google\.com/file/d/([\w-]+)/view\?usp=drive_link"
replacement = r"https://drive.google.com/uc?id=\1"

In [4]:
# ubah link attendance jd link spreadsheet masing2
link_airtable= 'https://drive.google.com/uc?id=1gIKKnVxugWQd_WxuOyGNSQRuPli_AOfw'
link_attendance = re.sub(pattern_link, replacement, insert_link)
link_level3= 'https://drive.google.com/uc?id=1JrZp_Ve6mekR-dPqR8xQBAlrgZtBjXqw'

Kalau error, kemungkinan bentuk linknya masih kaya gini:
https://drive.google.com/file/d/1gIKKnVxugWQd_WxuOyGNSQRuPli_AOfw/view?usp=drive_link
dihapus aja `/file/d/` dalam link diganti jadi `/uc?id=` , terus `/view?usp=drive_link` dihapus aja.

In [5]:
airtable = pd.read_csv(link_airtable)
attendance = pd.read_csv(link_attendance)
level3 = pd.read_csv(link_level3, skiprows=2)

# Clean Airtable

In [6]:
# ubah nama section
airtable_semarang = airtable[(airtable['Section'] == 'Semarang')]

In [7]:
# filter kolom yg dibutuhkan
airtable_name_email = airtable_semarang[['Student ID', 'Student Name', 'Email', 'Email RevoU']]

In [8]:
# rename column nama supaya jelas
airtable_name_email = airtable_name_email.rename(columns= {'Student ID':'id','Student Name': 'name', 'Email RevoU':'email_revou'})

In [9]:
# clean double space di nama
airtable_name_email['name'] = airtable_name_email['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
airtable_name_email['name'] = airtable_name_email['name'].str.upper()

<ipython-input-9-accc58cb7812>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  airtable_name_email['name'] = airtable_name_email['name'].str.strip().str.replace(r'\s+', ' ')


In [10]:
# ubah data type Student ID supaya gampang di merge
airtable_name_email['id'] = airtable_name_email['id'].astype('int64')

# Clean Attendance Zoom

In [11]:
# rename column nama supaya jelas
attendance_name_email = attendance.rename(columns= {'Name (Original Name)':'name','User Email': 'email', 'Duration (Minutes)':'duration'})

In [12]:
# filter kolom yg dibutuhkan
attendance_name_email = attendance_name_email[['name', 'email', 'duration']]

In [13]:
# clean double space di nama
attendance_name_email['name'] = attendance_name_email['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
attendance_name_email['name'] = attendance_name_email['name'].str.upper()

<ipython-input-13-6feaf5f74b62>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  attendance_name_email['name'] = attendance_name_email['name'].str.strip().str.replace(r'\s+', ' ')


# Clean Level 3

In [14]:
level3.dropna(subset=['Student ID'], inplace=True)

In [15]:
# ubah data type supaya gampang di merge
level3['Student ID'] = level3['Student ID'].astype('int64')
level3['Team'] = level3['Team'].astype('int64')

In [16]:
# rename column nama supaya jelas
level3 = level3.rename(columns= {'Student ID':'id','Student Name': 'name', 'Email':'email'})

In [17]:
# clean double space di nama
level3['name'] = level3['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
level3['name'] = level3['name'].str.upper()

<ipython-input-17-8e8b720d35b6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  level3['name'] = level3['name'].str.strip().str.replace(r'\s+', ' ')


In [18]:
# filter kolom yg dibutuhkan
absensi = level3.iloc[:, [0, 1, 2, 3, -1]]

In [19]:
absensi = absensi[['Team','name','email']]

## Check email kalau udah sama dengan airtable

In [20]:
dum_lv3= level3[['id','email']]

In [21]:
dum_email_air = airtable_name_email[['id','name', 'email_revou']]

In [22]:
check = pd.merge(dum_email_air, dum_lv3, on='id', how='left', suffixes = ('_airtable', '_level3'))

In [23]:
check.head()

,id,name,email_revou,email
0,7458741,DEVIANA PUSPITA,devpus89@gmail.com,devpus89@gmail.com
1,7460303,VIRGO JUNIOR SITEPU,virgojuniorsitepu@gmail.com,virgojuniorsitepu@gmail.com
2,7470736,FAURIKA,safaurika@gmail.com,safaurika@gmail.com
3,7471297,ARIANTO TAMBUNAN,tambunanarianto4@gmail.com,tambunanarianto4@gmail.com
4,7478722,FIRYAL AZKIATUL IFTINAN,firyalazkiatul04@gmail.com,firyalazkiatul04@gmail.com


In [24]:
check['Notes'] = check['email_revou'] == check['email']

In [25]:
mismatched_email = check[check['Notes'] == False]

In [26]:
mismatched_email

,id,name,email_revou,email,Notes


Kalau sama berarti aman :) Lanjut step berikut

# Next Step

In [27]:
attendance_name_email.shape

(1019, 3)

In [28]:
attendance_name_email['email'].fillna('-', inplace=True)

In [29]:
total_duration = attendance_name_email.groupby(['name', 'email'], as_index=False)['duration'].sum()

In [30]:
att_check = pd.merge(absensi, total_duration, how='outer', on = 'email', suffixes= ('_level3', '_zoom'))

In [31]:
att_check.shape

(361, 5)

In [32]:
condition_1 = (att_check['name_level3'].isnull()) & (att_check['name_zoom'].notnull())
hadir_zoom_ga_kedetect = att_check[condition_1]
hadir_zoom_ga_kedetect = hadir_zoom_ga_kedetect[['name_zoom', 'duration']]

In [33]:
condition_2 = (att_check['name_level3'].notnull()) & (att_check['name_zoom'].isnull())
yg_mau_dimerge = att_check[condition_2]
yg_mau_dimerge = yg_mau_dimerge[['Team','name_level3','email']]

In [34]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Yang join ZOOM

In [35]:
right = pd.merge(yg_mau_dimerge, hadir_zoom_ga_kedetect, left_on = 'name_level3', right_on= 'name_zoom', how='right')

In [36]:
condition_3 = (right['name_level3'].notnull()) & (right['email'].notnull()) & (right['name_zoom'].notnull())
fix = right[condition_3]

# TAMBAH PRESENSI (YG GA SESUAI EMAIL TAPI SAMA NAMA) AKA MATCH BY NAME YG DIRUASINYA >=60

In [37]:
# @title
sorted_fix = fix.sort_values(by='duration')
hadir_diatas_60 = sorted_fix[sorted_fix['duration'] >= 60]

In [38]:
hadir_diatas_60.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
20,6.0,MUHAMMAD RAMDANI YUSUF,ramdaniyusuf69@gmail.com,MUHAMMAD RAMDANI YUSUF,76.0
14,22.0,FARRELL ABRAR RHIZNANDA,farrellabrar18@gmail.com,FARRELL ABRAR RHIZNANDA,61.0


# TIDAK HADIR (GADA EMAIL TAPI SAMA NAMA) <60 DURASI

In [39]:
# @title
hadir_dibawah_60 = sorted_fix[sorted_fix['duration'] < 60]

In [40]:
hadir_dibawah_60.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
10,4.0,ALIFA BAIDURI HAYATUNNUFUS,alifabaiduri7@gmail.com,ALIFA BAIDURI HAYATUNNUFUS,44.0
16,19.0,FATIMAH HAFIAH,fatimahafiah69@gmail.com,FATIMAH HAFIAH,11.0


# Yg harus di RO (Merge program di Spreadsheet dan kode di atas)

In [41]:
# @title
a = att_check[~(condition_1 | condition_2)]
b = pd.concat([a, fix], ignore_index=True)

In [42]:
print(f'jumlah student yg berhasil ke detect: {b.shape[0]} akun. \n Note: ini hanya student. Tidak termasuk SM dan Mentor.')

jumlah student yg berhasil ke detect: 288 akun. 
 Note: ini hanya student. Tidak termasuk SM dan Mentor.


In [43]:
# @title
# yg di bawah 60 menit:
sorted_fix = b.sort_values(by='duration')
filtered_fix = sorted_fix[sorted_fix['duration'] < 60]

In [44]:
print(f'jumlah student yg harus di RO buat makesure: {filtered_fix.shape[0]} student.')

jumlah student yg harus di RO buat makesure: 55 student.


In [45]:
filtered_fix.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
6,1.0,UDWI ROSE JENE JANNAH,udwirose@gmail.com,AK_UDWIR.JENE,54.0
3,1.0,MUHAMMAD FATAN BASAR,fatanbasar@gmail.com,MUHAMMAD FATAN BASAR,52.0
2,1.0,MUHAMMAD FATAN BASAR,fatanbasar@gmail.com,FATAN,21.0
17,2.0,MUHAMMAD RADHI ALIFIANSYAH,alifmhow22@gmail.com,MUHAMMAD RADHI ALIFIANSYAH,58.0
13,2.0,ADIT PRAYOGA,aditprayogaa02@gmail.com,ADIT PRAYOGA,28.0
284,4.0,ALIFA BAIDURI HAYATUNNUFUS,alifabaiduri7@gmail.com,ALIFA BAIDURI HAYATUNNUFUS,44.0
36,4.0,ALLIF RIZKI ABDILLAH,allifrizki02@gmail.com,DILLAH,54.0
45,5.0,WIDYA RACHMANIA,widyarch@gmail.com,WIDYA -,21.0
52,5.0,ANTI OKTAVIANI,antioktaviani32@gmail.com,5_ANTI OKTAVIANI,11.0
46,5.0,WIDYA RACHMANIA,widyarch@gmail.com,WIDYA RACHMANIA,54.0


In [46]:
# @title
def check_name_di_merge(name):
  contains_name = (filtered_fix['name_zoom'].str.contains(name, na=False)) | (filtered_fix['name_level3'].str.contains(name, na=False))
  return filtered_fix[contains_name]

In [47]:
# buat make sure di check aja namanya ada ga
result = check_name_di_merge('ANGELINE')
result

,Team,name_level3,email,name_zoom,duration


# Yang Ga Ke Detect

In [48]:
# @title
outer = pd.merge(yg_mau_dimerge, hadir_zoom_ga_kedetect, left_on = 'name_level3', right_on= 'name_zoom', how='outer')

In [49]:
# @title
condition_4 = (outer['name_level3'].notnull()) & (outer['email'].notnull())
f = outer[~(condition_4)]

In [50]:
f.shape

(24, 5)

In [51]:
f

,Team,name_level3,email,name_zoom,duration
49,NaN,NaN,NaN,.,1.0
50,NaN,NaN,NaN,04_ALIFA BAIDURI HAYATUNNUFUS,11.0
51,NaN,NaN,NaN,04_ALIFA BAIDURI HAYATUNNUFUS (ALIFA BAIDURI H...,47.0
52,NaN,NaN,NaN,15_ATIKA INDAH MENTARI,54.0
53,NaN,NaN,NaN,15_ATIKA INDAH MENTARI (ATIKA INDAH MENTARI),46.0
54,NaN,NaN,NaN,16_HIZBULLAH,56.0
55,NaN,NaN,NaN,16_HIZBULLAH (PCHOLDER),40.0
56,NaN,NaN,NaN,22_FARRELL ABRAR RHIZNANDA,10.0
57,NaN,NaN,NaN,22_FARRELL ABRAR RHIZNANDA (FARRELL ABRAR RHIZ...,79.0
58,NaN,NaN,NaN,AKMAL,129.0


In [52]:
pattern = r'^\d+_([A-Z\s]+)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern, r'\1', regex=True)
pattern2 = r'^\d+\s+([A-Z\s]+)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern2, r'\1', regex=True)

<ipython-input-52-5ffb9168a6e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern, r'\1', regex=True)
<ipython-input-52-5ffb9168a6e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern2, r'\1', regex=True)


In [53]:
total_duration1 = f.groupby(['name_zoom'], as_index=False)['duration'].sum()

In [54]:
f.dropna(subset=['name_zoom', 'duration'], inplace=True)

<ipython-input-54-a0f4cd478d51>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.dropna(subset=['name_zoom', 'duration'], inplace=True)


In [55]:
pattern3 = r'\((.*?)\)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern3, '', regex=True).str.strip()

<ipython-input-55-40ad52d32664>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern3, '', regex=True).str.strip()


In [56]:
total_duration1 = f.groupby(['name_zoom'], as_index=False)['duration'].sum()

In [57]:
total_duration1

,name_zoom,duration
0,.,1.0
1,AGIT,163.0
2,AKMAL,129.0
3,ALIFA BAIDURI HAYATUNNUFUS,58.0
4,AMINATUS ZAHRO,16.0
5,ANGELINE LYDIA,9.0
6,ARGA ADYATAMA,148.0
7,ATIKA INDAH MENTARI,100.0
8,FARRELL ABRAR RHIZNANDA,89.0
9,FATIMAH FITRIA,3.0


In [58]:
f

,Team,name_level3,email,name_zoom,duration
49,NaN,NaN,NaN,.,1.0
50,NaN,NaN,NaN,ALIFA BAIDURI HAYATUNNUFUS,11.0
51,NaN,NaN,NaN,ALIFA BAIDURI HAYATUNNUFUS,47.0
52,NaN,NaN,NaN,ATIKA INDAH MENTARI,54.0
53,NaN,NaN,NaN,ATIKA INDAH MENTARI,46.0
54,NaN,NaN,NaN,HIZBULLAH,56.0
55,NaN,NaN,NaN,HIZBULLAH,40.0
56,NaN,NaN,NaN,FARRELL ABRAR RHIZNANDA,10.0
57,NaN,NaN,NaN,FARRELL ABRAR RHIZNANDA,79.0
58,NaN,NaN,NaN,AKMAL,129.0


In [59]:
# run this to check if there's some name you want to check
def check_name(name):
  contains_name = (f['name_zoom'].str.contains(name, na=False)) | (filtered_fix['name_zoom'].str.contains(name, na=False)) | (hadir_diatas_60['name_zoom'].str.contains(name, na=False))
  return f[contains_name]

In [60]:
# and this, just change the name
result = check_name('TIFFANY')
result

<ipython-input-59-37d14bf88415>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return f[contains_name]


,Team,name_level3,email,name_zoom,duration
